### Oppgave 1: Finne landene med høyeste og laveste befolkningstetthet

For å løse oppgaven må vi først finne datasettene med data om landenes innbyggertall og areal.

Deretter kobler vi datasettene. Vi velger fjerne land med missing eller 0 i antall innbyggere eller areal. Vi kunne ha valgt å imputere fra et annet datasett (se logisk imputering under [2. Funksjoner i klargjøring](https://jupyter.prod-bip-app.ssb.no/hub/user-redirect/lab/tree/kurs-ssb-dapla-veileder/notebooks/2.%20Funksjoner%20i%20klargj%C3%B8ring/5.%20Logisk%20imputering.ipynb)).

I tillegg fjerner vi dubletter og velger ut hvilke observasjoner vi skal beholde.

Når datasettet er slått sammen, lager vi variabelen <code>befolkningstetthet</code>, og finner så høyeste og laveste verdi for denne variabelen.

#### Laster inn bibliotek:

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

#### 1.1  Finne og leses inn datasett

In [ ]:
# lete etter datasettene om innbyggerantall 2020 og areal
spark.read.path("/felles/veiledning/pyspark/eksempler*").show(10, False)

In [ ]:
# lese inn datasettet om landenes innbyggerantall i 2020
df_areal = spark.read.path("/felles/veiledning/pyspark/eksempler/areal")

In [ ]:
# lese inn datasettet om landenes areal
df_innbyggerantall_2020 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")

In [ ]:
# skrive ut dataene
print("*********************************** AREAL ***********************************")
df_areal.show(50,False)
print("*********************************** INNBYGGERANTALL 2020 ***********************************")
df_innbyggerantall_2020.show(50,False)

#### 1.2 Fjerne observasjoner som er 0
Fjerne observasjoner som er 0 for variablene <code>Innbyggerantall</code> (Dvs. Færøyene som vi ser i resultatet over)

In [ ]:
# Filtrerer de observasjonenen hvor Innbyggerantall>0
df_innbyggerantall_2020 = df_innbyggerantall_2020.filter(df_innbyggerantall_2020.Innbyggerantall>0)
df_innbyggerantall_2020.show(50, False)

#### 1.3 Fjerne dubletter
Vi ønsker å beholde de dublettene som har høyste verdi for Innbyggerantall og laveste verdi for areal. Det kan vi gjøre ved først å sortere datasettet.

In [ ]:
#Sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
df_innbyggerantall_2020_sort = df_innbyggerantall_2020.orderBy(['Land','Innbyggerantall'], ascending=[1, 0])
print("*********************************** INNBYGGERANTALL SORTERT ETTER LAND OG INNBYGGERANTALL ***********************************")
df_innbyggerantall_2020_sort.show(50,False)

In [ ]:
#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_land_dub_alle  = df_innbyggerantall_2020_sort.join(df_innbyggerantall_2020_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')
print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_land_dub_alle.show()

In [ ]:
# Fjerner duplikater (flere observasjoner på land)
print("*********************************** INNBYGGERANTALL UTEN DUBLIKATER. HØYESTE INNBYGGERANTALL ER BEHOLDT ***********************************")
df_innbyggerantall_udub= df_innbyggerantall_2020_sort.drop_duplicates(subset=['Land'])
df_innbyggerantall_udub.show(50,False)

In [ ]:
#Sorterer etter land og areal og lister ut den sorterte dataframen
print("*********************************** AREALDATASETT SORTERT ETTER LAND OG AREAL ***********************************")

df_areal_sort = df_areal.orderBy(['Land','Areal'], ascending=[1, 1])
df_areal_sort.show(50,False)

#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_areal_dub_alle  = df_areal_sort.join(df_areal_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')

print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_areal_dub_alle.show()  

print("*********************************** AREALDATSETTET UTEN DUBLETTER: MINSTE AREAL BEHOLDT ***********************************")
df_areal_udub= df_areal_sort.drop_duplicates(subset=['Land'])
df_areal_udub.show(50,False)

#### 1.4 Koble datasett
Vi bruker koblingsmetoden 'inner' fordi vi ikke er interessert i forekomster som har missing i enten Innbyggerantall eller Areal.

In [ ]:
# Koble sammen datasettene (innbyggerantall og areal). .
df_areal_innbyggerantall = df_areal_udub.join(df_innbyggerantall_udub, 'Land', 'inner')

print("*********************************** SAMMENKOBLET DATASETT ***********************************")
df_areal_innbyggerantall.show(50,False)

#### 1.5 Lage en ny variabel

In [ ]:
# Ny variabel for befolkningstetthet
df_areal_innbyggerantall = df_areal_innbyggerantall.withColumn('Befolkningstetthet',(df_areal_innbyggerantall['Innbyggerantall']/df_areal_innbyggerantall['Areal']).cast(IntegerType()))

print("*********************************** SAMMENKOBLET DATASETT MED NY VARIABEL ***********************************")
df_areal_innbyggerantall.show(50,False)

1.5 Finne maks og min

In [ ]:
# Finne maks og min av befokningstetthet
result = df_areal_innbyggerantall.select([F.min("Befolkningstetthet"), F.max("Befolkningstetthet")])

print("*********************************** MINIMUM OG MAKSIMUM AV VARIABELEN BEFOLKNINGSTETTHET ***********************************")
result.show()
